In [ ]:
import numpy as np
import glob
from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm, ListedColormap
import seaborn as sns
from scipy.stats import sigmaclip, binned_statistic as bs
from scipy.spatial import KDTree
from astropy.stats import sigma_clip
import pandas as pd
from multiprocessing import Pool
import time
import math
import skimage.morphology
from matplotlib.ticker import MaxNLocator
from matplotlib.backends.backend_pdf import PdfPages
import astropy.time
%matplotlib inline

In [ ]:
def sigepoch(epoi):
    epois = [epoi]
    if(epoi != 8):
        epois.append(epoi+1)
    return tuple(set(np.linspace(0, 7, 8, dtype=int)) - set(epois))

In [ ]:
datagroup = 'David'

detdf = pd.DataFrame()
detdf['Galaxy'] = None
detdf['Epoch'] = None
detdf['XPix'] = None
detdf['YPix'] = None
detdf['Date'] = None
detdf['Sigma'] = None

ddc = 0 #detdfcounter = ddc

s = np.genfromtxt('{}_centroids.txt'.format(datagroup), dtype=str, delimiter=',')
if(datagroup == 'David'):
    galaxies = [blah[0].split('mosaic_')[1].split('-1_sub.fits')[0] for blah in s]
else:
    galaxies = [blah[0].split('/')[-1].split('_')[0] for blah in s]

c = 8.461595E-6
siglevel = 20
fo = 280.9

#galaxies = ['Arp_148']

#boxsize = 30 # This one is for the fake supernova but it goes out farther
boxsize = int(10/0.6)/2 # This one is for the fake sne, out to 10 arcsecs
box = 20 # This one is for Cropping/subarray (pixels)

for iter_gal, galaxy in enumerate(galaxies):
    print('*****Doing {}*****'.format(galaxy))
    galstart = time.time()
    g = glob.glob('{}_data/*{}*.fits'.format(datagroup, galaxy))
    subimages = np.zeros((box*2,box*2,8), dtype=float)

    cents = s[iter_gal,1:].astype(float)
    centx = cents[0]
    centy = cents[1]
    icx = int(centx)
    icy = int(centy)

    for epoi in range(8):
        datg = '{}_data/'.format(datagroup)
        if(datagroup == 'David'):
            fname = '{}mosaic_{}-{}_sub.fits'.format(datg, galaxy, epoi+1)
        else:
            fname = '{}{}_epoch{}_aligned.fits'.format(datg,galaxy,epoi+1)
        dat = fits.getdata(fname, ext=0)
        
        subimages[:,:,epoi] = dat[icy-box:icy+box, icx-box:icx+box]

    med = np.median(subimages, axis=2)

    subminmed = np.zeros_like(subimages)
    for x in range(subimages.shape[2]):
        subminmed[:,:,x] = subimages[:,:,x] - med

    pixdist = np.vstack(np.meshgrid(np.arange(box*2)-(box), np.arange(box*2)-(box))).reshape(2,-1).T
    dist = pixdist * 0.6
    dist = dist ** 2
    dist = np.sum(dist, axis=1)
    dist = np.sqrt(dist)
    B = np.reshape(dist, (box*2, box*2))

    #Plot everything as a fart plot across all 8 epochs
#         fig, axarr = plt.subplots(4,2)
#         fig.set_size_inches(30,50)
#         fig.set_dpi(100)

    plt.rcParams.update({'font.size': 24})
    colors = sns.color_palette('Set2', 20)
    sns.set(palette='Set1', font_scale=2.5)
    sns.set_style('darkgrid', {'axes.facecolor': '#F9F9F9', 'grid.color':'#DCDCDC'})
    sns.despine()
    
    with PdfPages('plots/nofsf/{}_{}_epochs_combined_smaller2.pdf'.format(galaxy, datagroup)) as pdf:
        fig, axarr = plt.subplots(4,2)
        fig.set_size_inches(30,60)
        fig.set_dpi(50)
        axs = axarr.reshape(-1)
        
        for epoi in range(8):

            ax = axs[epoi]
            std = np.std(subminmed[:,:,sigepoch(epoi)], axis=2)
            labeled, labnum = skimage.morphology.label(subminmed[:,:,epoi] > std*siglevel, neighbors=8, return_num=True)
            coords_yx = { i: (labeled == i).nonzero() for i in range(1,labnum+1) if len((labeled == i).nonzero()[0]) > 2}

            for epoch in range(8):
                d = subminmed[:,:,epoch] * c * 1E3
                ax.plot(B.reshape(-1), d.reshape(-1), 'o', color=colors[epoch], label='Epoch {}'.format(epoch+1),
                        markersize=12, alpha=0.375)
            ax.plot(B.reshape(-1), std.reshape(-1)*siglevel*c*1E3, '.', color='black', label='3$\sigma$', alpha=0.35)
            ax.plot(B.reshape(-1), std.reshape(-1)*-3*c*1E3, '.', color='black', alpha=0.35)
            di = B.reshape(-1)
            st = std.reshape(-1)*siglevel*c*1E3

            for ikey, key in enumerate(list(coords_yx.keys())):
                y = coords_yx[key][0]
                x = coords_yx[key][1]
                vdist = []
                sigs = []
                for coo in range(len(x)):
                    vdist.append(B[y[coo], x[coo]])
                ax.vlines(np.mean(vdist), min(-1*st), max(st), colors=colors[ikey])
                #ax.vlines(np.mean(vdist), -0.002, 0.002, colors=colors[ikey])

            ax.set_xlabel('Distance from center (arcsec)')
            ax.set_ylabel('Residual Flux (mJy)')
            ax.legend(loc=3, ncol=2)
            ax.grid(alpha=0.25)
            ax.set_title('Epoch {}'.format(epoi+1))
            fig.suptitle('{} - {} Sword plots'.format(galaxy, datagroup))
        pdf.savefig(fig)
        plt.close()
        
        fig, axarr = plt.subplots(4,2)
        fig.set_size_inches(30,60)
        fig.set_dpi(50)
        axs = axarr.reshape(-1)
        for epoi in range(8):
            ax = axs[epoi]
            std = np.std(subminmed[:,:,sigepoch(epoi)], axis=2)
            labeled, labnum = skimage.morphology.label(subminmed[:,:,epoi] > std*siglevel, neighbors=8, return_num=True)
            coords_yx = { i: (labeled == i).nonzero() for i in range(1,labnum+1) if len((labeled == i).nonzero()[0]) > 2}
            ax.imshow(subimages[:,:,epoi], origin=0, alpha=1, vmin=-1, vmax=1, cmap='Greys_r')
            
            for ikey, key in enumerate(list(coords_yx.keys())):
                y = coords_yx[key][0]
                x = coords_yx[key][1]
                detdf.at[ddc,'Galaxy'] = galaxy
                detdf.at[ddc,'Epoch'] = epoi+1
                detdf.at[ddc,'XPix'] = x
                detdf.at[ddc,'YPix'] = y
                detdf.at[ddc,'Sigma'] = np.max(subminmed[x,y,epoi]/std[x, y])
                ddc += 1
                for coo in range(len(x)):
                    ax.plot(x[coo],y[coo], 'o', color=colors[ikey], markersize=5)
            #ax.imshow(subminmed[:,:,epoi] > std*siglevel, origin=0)
            ax.grid(alpha=0.25)
            ax.set_title('Epoch {}'.format(epoi+1))
        fig.suptitle('{} - {} Data'.format(galaxy, datagroup))
        pdf.savefig(fig)
        plt.close()